In [2]:
!pip install pyspark



     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 37.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=461b7351c286481da55a9762a507d6a3cd3ba37747a48b75dd529918a6186c30
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
import collections

from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql import SparkSession

import re
from bs4 import BeautifulSoup
import requests
import random
import re
import numpy as np
import pandas as pd
import time
import random

In [4]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wine_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/wine.data').drop('Unnamed: 0', axis=1)
wine_data

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [6]:
# No Pyspark
#Euclid for each row = 178x178
start = time.time()
from scipy.spatial.distance import cdist
d = cdist(wine_data, wine_data, 'euclid')
df = pd.DataFrame(data=d)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177
0,0.000000,31.265012,122.831154,415.245400,330.174500,385.295960,227.129474,230.091599,36.116966,35.274977,445.551026,217.378539,257.819328,92.418667,482.666943,245.474259,215.163965,66.270697,615.303691,220.282747,285.002376,296.086538,39.763341,59.456228,222.229877,235.222466,134.383524,222.481210,151.382550,43.161300,221.642847,450.506927,78.496266,170.125075,34.718304,147.578043,185.791609,49.531192,53.609584,305.020527,...,486.600394,475.801740,466.082026,287.581831,545.899593,516.229294,210.620685,236.433366,651.736447,441.977041,416.543392,515.256293,565.097523,585.262842,640.182387,391.138175,425.725011,341.802788,586.270837,187.820960,406.221579,446.725072,546.442440,385.565817,495.503657,390.600076,451.598616,546.458621,370.476539,382.068561,315.927567,435.392211,555.895155,596.455105,406.657723,326.660049,316.110527,230.240023,225.215184,506.059368
1,31.265012,0.000000,135.224693,430.251561,315.668800,400.210294,240.063413,245.984974,6.786383,7.832918,460.082389,230.127051,270.271069,100.441396,497.020156,260.364064,231.048759,81.911220,630.090775,205.669318,271.297799,280.114035,16.011218,35.976794,205.231222,221.740499,145.253606,235.150226,135.434744,16.295423,235.283179,465.113878,60.443740,187.936088,46.762419,130.336307,170.350997,55.465636,30.391704,291.368267,...,470.241503,460.188582,450.134975,270.386179,530.161580,500.157903,195.575941,220.256185,635.395420,425.387145,400.245661,500.312706,550.654490,570.272138,625.380071,375.141374,410.130816,325.306227,570.197485,170.819068,390.354821,430.360370,530.247937,370.188581,480.171007,375.135731,435.292068,530.276966,355.438694,365.354470,300.381312,420.435462,540.084013,580.264085,390.251934,310.236117,300.274500,216.221232,211.213539,490.235268
2,122.831154,135.224693,0.000000,295.262711,450.331053,265.256948,105.211462,111.827681,140.149239,140.074677,325.027220,95.232444,135.565249,37.154720,362.069296,125.511709,96.905700,56.802272,495.066674,340.351467,405.782089,415.007439,150.034327,170.127297,340.050394,355.810280,13.167619,100.293659,270.074192,150.119458,100.083656,330.042938,195.041765,58.868154,90.477629,265.012246,305.155618,80.093912,165.080132,425.896789,...,605.153986,595.035727,585.039641,405.198629,665.043553,635.080874,330.216808,355.035132,770.305991,560.228685,535.102605,635.142082,685.389131,705.120500,760.186934,510.043371,545.023062,460.110074,705.120911,305.170138,525.105574,565.180251,665.145157,505.052610,615.041020,510.039789,570.142629,665.153706,490.160485,500.205923,435.081395,555.167072,675.031791,715.182277,525.125384,445.076785,435.042939,350.571188,345.562652,625.070178
3,415.245400,430.251561,295.262711,0.000000,745.039443,30.091969,190.796230,185.204752,435.313485,435.261604,31.159418,200.840403,161.816385,330.788196,68.076464,170.012873,200.159475,350.025760,200.067991,635.017745,700.125318,710.101375,445.183130,465.372051,635.252973,650.162072,285.722591,195.977448,565.049043,445.339654,195.468747,35.809622,490.102628,245.768432,385.040988,560.171865,600.021443,375.327140,460.271350,720.173853,...,900.365902,890.124627,880.191710,700.436158,960.173849,930.261471,625.023234,650.134961,1065.536259,855.454588,830.290159,930.024822,980.086664,1000.027560,1055.047099,805.157639,840.078892,755.302015,1000.302374,600.317833,820.199919,860.369235,960.345882,800.043381,910.055841,805.050199,865.334565,960.356662,785.044589,795.415505,730.096273,850.053083,970.167815,1010.378276,820.313787,740.249375,730.123558,645.068999,640.062584,920.201042
4,330.174500,315.668800,450.331053,745.039443,0.000000,715.054595,555.475848,560.020404,310.797076,310.702995,775.117512,545.504306,585.742194,415.996072,812.216512,575.052483,545.009575,395.021928,945.075834,110.177836,4

In [7]:
#find top 3 min value in each row
from heapq import nsmallest
df1 =  pd.DataFrame(data = [nsmallest(4, df.iloc[i,:]) for i in range(0,df.count()[0])]).drop(0, axis=1)
df1

,1,2,3
0,10.392805,22.340748,24.760232
1,6.786383,7.832918,13.140765
2,12.296992,13.167619,36.710573
3,30.091969,31.159418,35.809622
4,17.728680,20.776525,22.421559
...,...,...,...
173,12.599016,15.058277,15.235862
174,3.885010,11.312162,12.599016
175,5.358890,11.140767,13.301038
176,5.358890,9.599422,13.516649


In [8]:
#find index min 
df3 = pd.DataFrame(data =[[df.iloc[i].tolist().index(j) for j in df1.iloc[i].tolist()] for i in range(0, len(df1.index))], columns=['neighbor_index_1','neighbor_index_2','neighbor_index_3']).reset_index()
df3

,index,neighbor_index_1,neighbor_index_2,neighbor_index_3
0,0,54,45,48
1,1,8,9,48
2,2,52,26,50
3,3,5,10,31
4,4,68,168,174
...,...,...,...,...
173,173,174,168,155
174,174,168,68,173
175,175,176,25,19
176,176,175,19,25


In [9]:
end = time.time()
print(end - start)

0.537118673324585


In [10]:
#Pyspark
#convert dataframe to pyspark datadrame
# spark = SparkSession.builder.master("local[1]").appName("Euclid").getOrCreate()
# sparkDF=spark.createDataFrame(wine_data) 
# sparkDF.printSchema()
# sparkDF.show()


In [11]:
conf = SparkConf().setMaster("local[*]").setAppName("Euclid")
sc = SparkContext.getOrCreate(conf=conf)
# rdd1 = sc.parallelize(wine_data.values.tolist())
# rdd1.collect()


In [12]:
# rdd2 = rdd1.cartesian(rdd1)
# rdd2.collect()

In [13]:
from scipy.spatial import distance
from heapq import nsmallest
def min_Euclid(x,array):
  array_new = []
  for i in array:
    array_new.append(distance.euclidean(x, i))
  all_euclib_top3 =  nsmallest(4,array_new)
  del all_euclib_top3[0]
  return all_euclib_top3, array_new
def find_index(x):
  array = []
  for i in x[0]:
    array.append(x[1].index(i))
  return array


In [14]:
start = time.time()
rdd = sc.parallelize(wine_data.values.tolist())
rdd1 = rdd.zipWithIndex()
array = rdd.collect()

rdd2 = rdd1.map(lambda x: (x[1], x[0], array))
rdd2.collect()




Output hidden; open in https://colab.research.google.com to view.

In [15]:
rdd3 = rdd2.map(lambda x: (x[0],min_Euclid(x[1],x[2])))
rdd3.collect()



[(0,
  ([10.392805203601192, 22.340747525541754, 24.760232228313207],
   [0.0,
    31.265012394048398,
    122.83115402860953,
    415.24539985411036,
    330.1745000753389,
    385.2959598801939,
    227.12947364884198,
    230.09159915129453,
    36.116965542525854,
    35.27497696668277,
    445.55102603405595,
    217.3785391891297,
    257.81932782473854,
    92.4186669455906,
    482.66694272551956,
    245.47425934301137,
    215.16396491977926,
    66.2706971443639,
    615.3036905463838,
    220.28274694128908,
    285.00237595500846,
    296.0865376878861,
    39.763341157402756,
    59.45622843739754,
    222.22987737925789,
    235.22246597636035,
    134.38352354362493,
    222.48120976837572,
    151.38254952272405,
    43.161299795071045,
    221.6428467151602,
    450.506927027765,
    78.49626551626517,
    170.125075165303,
    34.71830352998257,
    147.57804274349218,
    185.79160933691273,
    49.53119219239529,
    53.60958403121591,
    305.0205271780901,
    27

In [16]:
rdd4 = rdd3.map(lambda x: (x[0], find_index(x[1]) ))
rdd4.collect()
rdd4 = rdd4.map(lambda x: (x[0], x[1][0], x[1][1], x[1][2]))
rdd4DF = pd.DataFrame(data = rdd4.collect(), columns=['index','neighbor_index_1','neighbor_index_2','neighbor_index_3']);
rdd4DF

,index,neighbor_index_1,neighbor_index_2,neighbor_index_3
0,0,54,45,48
1,1,8,9,48
2,2,52,26,50
3,3,5,10,31
4,4,68,168,174
...,...,...,...,...
173,173,174,168,155
174,174,168,68,173
175,175,176,25,19
176,176,175,19,25


In [17]:
end = time.time()
print(end - start)

12.481360912322998
